In [1]:
# Dependencies.
import pandas as pd
from config import client_id, client_secret
from params import key_signatures, modes, key_int
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from pprint import pprint
import numpy as np

# Load variables to environment.
os.environ['SPOTIPY_CLIENT_ID'] = client_id
os.environ['SPOTIPY_CLIENT_SECRET'] = client_secret

In [2]:
# Function to obtain user's input.
def get_input():

    user_input = input('Enter the playlist URI: ')

    return user_input

In [3]:
# Function to perform API call and return results in JSON format.
def get_results(t, o):

    sp = spotipy.Spotify(client_credentials_manager = SpotifyClientCredentials())

    # Check if string contains 'https', the ref['next'] will contain this if the list has more than 100 songs.
    if 'https' in t:

        # Split the string in URL using '/'
        split = t.split('/')

        # Split list index 5 will contain part of URI, multiple o by 100 for the offset in the function call.
        results = sp.playlist_tracks('spotify:playlist:' + split[5], offset = o * 100)

    else:
        results = sp.playlist_tracks(t)

    return results

In [4]:
# Function to create dict with data.
def create_df(results):

    # Empty dict to hold data from Spotify.
    data = {
        'track': [],
        'artist': [],
        'uri': []
    }

    # Loop control.
    loop = True

    # Results as json object.
    ref = results

    # Offset variable.
    offset = 0

    while loop:

        # Adds values to dict.
        for song in ref['items']:
            data['track'].append(song['track']['name'])
            data['artist'].append(song['track']['artists'][0]['name'])
            data['uri'].append(song['track']['uri'])

        # Check for ref['next'], if there's text then playlist has more items.
        if ref['next'] is not None:
            offset += 1
            ref = get_results(ref['next'], offset)
        
        # Terminate loop.
        else:
            loop = False

    return pd.DataFrame(data)

In [5]:
# Function to perform API call for audio features and return results in JSON format.
def get_features(t):

    sp = spotipy.Spotify(client_credentials_manager = SpotifyClientCredentials())

    results = sp.audio_features(t)

    return results

In [6]:
# Function to find the time key signatures of the song.
def features_finder(results):

    try:

        m = results[0]['mode']

        k = results[0]['key']

        t = results[0]['time_signature']

        b = results[0]['tempo']
    
    except IndexError:
        
        return np.NaN, np.NaN, np.NaN, np.NaN
    
    return m, k, t, b

In [7]:
results = get_results(get_input(), 1)
df = create_df(results)

uris = list(df['uri'])

md_col = []
ks_col = []
kint_col = []
ts_col = []
bpm_col = []

for uri in uris:
    response = get_features(uri)
    md, ks, ts, bpm = features_finder(response)

    md_col.append(md)
    ks_col.append(ks)
    kint_col.append(key_int[md][ks])
    ts_col.append(ts)
    bpm_col.append(bpm)


df['mode'] = md_col
df['key_signature'] = ks_col
df['key_integer'] = kint_col
df['time_signature'] = ts_col
df['bpm'] = bpm_col


df = df.dropna()
df

,track,artist,uri,mode,key_signature,key_integer,time_signature,bpm
0,Don't Stop Believin',Journey,spotify:track:77NNZQSqzLNqh2A9JhLRkg,1,4,4,4,119.250
1,Any Way You Want It,Journey,spotify:track:2DyHhPyCZgZzNXn1IrtsTu,1,7,1,4,138.162
2,Separate Ways (Worlds Apart),Journey,spotify:track:2ORySgkn0CbUMFbEoP1wwE,0,4,1,4,131.446
3,Only the Young,Journey,spotify:track:5DkORVajfAFyI6PJnFFfng,1,7,1,4,72.479
4,Ask the Lonely,Journey,spotify:track:1YVyN4CV9VZGxZNL2PDvpw,1,0,0,4,128.396
...,...,...,...,...,...,...,...,...
307,Rocket - Remastered 2017,Def Leppard,spotify:track:3HtthIc2aeQzzoffjjw9S5,1,11,5,4,81.985
308,Hysteria - Remastered 2017,Def Leppard,spotify:track:772K2oHttS2C7zTCfL9mHC,1,7,1,4,106.997
309,Rock Of Ages,Def Leppard,spotify:track:3RnYLgMnh9WOWfB2k0UQUB,1,4,4,4,105.837
310,Bringin' On The Heartbreak - Remastered 2018,Def Leppard,spotify:track:3I5BIp1ajVFFi5VpbIbcRO,0,9,0,4,78.031


In [8]:
df.to_csv('output.csv', index = False)